### Trabalho Titanic - Previsão de Sobreviventes

O trabalho consiste em , atraves do dataset disponibilizado 'titanic.csv', tentar prever se um passageiro sobreviveu ou não ao naufrágio.

In [1]:
# Importando as bibliotecas 
from datetime import datetime

print(f'Importando as bibliotecas - {datetime.now()}.')
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql import *
from pyspark.sql.functions import col, asc
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import * 
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

print(f'Importação das bibliotecas com sucesso - {datetime.now()}.')

Importando as bibliotecas - 2020-08-26 18:21:49.734433.
Importação das bibliotecas com sucesso - 2020-08-26 18:21:49.883029.


In [2]:
# Instalando Java JDK e Spark no Colaboratory.

print(f'Instalação do JAVA JDK e Spark-3.0.0-bin-hadoop2.7 iniciada - {datetime.now()}.')

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install -q pyspark

print(f'Instalação das aplicações com sucesso - {datetime.now()}.')

Instalação do JAVA JDK e Spark-3.0.0-bin-hadoop2.7 iniciada - 2020-08-26 18:21:53.027440.
Instalação das aplicações com sucesso - 2020-08-26 18:22:23.840271.


In [3]:
# Criando as variáveis de ambientes do JAVA e SPARK no  Colaboratory.

print(f'Criação das variáveis de ambiente iniciada- {datetime.now()}.')

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"

print(f'Criação das variáveis de ambiente com sucesso - {datetime.now()}.')

Criação das variáveis de ambiente iniciada- 2020-08-26 18:22:27.380440.
Criação das variáveis de ambiente com sucesso - 2020-08-26 18:22:27.380638.


In [4]:
# Criando uma aplicação "Titanic" na Sessão Spark.

print(f'Criação da Sessão Spark e Aplicativo Spark Titanic iniciada- {datetime.now()}.')

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Titanic") \
    .getOrCreate()

print(f'Criação da Sessão Spark e Aplicativo Spark Titanic com sucesso - {datetime.now()}.')

Criação da Sessão Spark e Aplicativo Spark Titanic iniciada- 2020-08-26 18:22:30.833493.
Criação da Sessão Spark e Aplicativo Spark Titanic com sucesso - 2020-08-26 18:22:36.331258.


In [5]:
# Apresentando a Sessão Spark Criada.

print(f'Apresentação da Sessão Spark e Aplicativo Spark Titanic iniciada- {datetime.now()}.')

print('')
print(spark)
print('')

print(f'Apresentação da Sessão Spark e Aplicativo Spark Titanic com sucesso - {datetime.now()}.')

Apresentação da Sessão Spark e Aplicativo Spark Titanic iniciada- 2020-08-26 18:22:40.231377.


Apresentação da Sessão Spark e Aplicativo Spark Titanic com sucesso - 2020-08-26 18:22:40.231637.


In [6]:
# Instanciando o Spark.

print(f'Instanciando Spark iniciada- {datetime.now()}.')

from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)

print(f'Instanciação Spark com sucesso - {datetime.now()}.')

Instanciando Spark iniciada- 2020-08-26 18:22:45.842778.
Instanciação Spark com sucesso - 2020-08-26 18:22:45.852576.


In [7]:
# Selecionar o arquivo titanic.csv e importar para Google Colaboratory.

print('Selecione o arquivo titanic.csv salvo no seu computador.')
print('')
print(f'Upload arquivo titanic.csv iniciada- {datetime.now()}.')

from google.colab import files
data_to_load = files.upload()

print(f'Upload arquivo titanic.csv com sucesso - {datetime.now()}.')

Selecione o arquivo titanic.csv salvo no seu computador.

Upload arquivo titanic.csv iniciada- 2020-08-26 18:22:51.189337.


Saving titanic.csv to titanic.csv
Upload arquivo titanic.csv com sucesso - 2020-08-26 18:22:57.819725.


In [8]:
# Importação do Dataset titanic.csv para o Dataframe df.

print(f'Importação do titanic.csv para o Dataframe df iniciada - {datetime.now()}.')

df = spark.read.csv('/content/titanic.csv', inferSchema=True, sep=',',encoding='ISO-8859-1',header=True)

print(f'Importação do titanic.csv para o Dataframe df com sucesso - {datetime.now()}.')

Importação do titanic.csv para o Dataframe df iniciada - 2020-08-26 18:23:03.431084.
Importação do titanic.csv para o Dataframe df com sucesso - 2020-08-26 18:23:09.531251.


In [9]:
# Apresentação das 20 primeiras linhas do Dataframe df.

print('Apresentação das 20 primeiras linhas do Dataframe df')
df.show()

Apresentação das 20 primeiras linhas do Dataframe df
+--------+------+--------------------+------+----+-----------------------+-----------------------+-------+
|Survived|Pclass|                Name|   Sex| Age|Siblings/Spouses Aboard|Parents/Children Aboard|   Fare|
+--------+------+--------------------+------+----+-----------------------+-----------------------+-------+
|       0|     3|Mr. Owen Harris B...|  male|22.0|                      1|                      0|   7.25|
|       1|     1|Mrs. John Bradley...|female|38.0|                      1|                      0|71.2833|
|       1|     3|Miss. Laina Heikk...|female|26.0|                      0|                      0|  7.925|
|       1|     1|Mrs. Jacques Heat...|female|35.0|                      1|                      0|   53.1|
|       0|     3|Mr. William Henry...|  male|35.0|                      0|                      0|   8.05|
|       0|     3|     Mr. James Moran|  male|27.0|                      0|                 

In [10]:
# Apresentação da Estrutura do Dataframe df.

print('Apresentação da Estrutura do Dataframe df')
display(df)

Apresentação da Estrutura do Dataframe df


DataFrame[Survived: int, Pclass: int, Name: string, Sex: string, Age: double, Siblings/Spouses Aboard: int, Parents/Children Aboard: int, Fare: double]

In [11]:
# Outra forma de apresentação da Estrutura do Dataframe df.

print('Outra forma de apresentação da Estrutura do Dataframe df')
df.printSchema()

Outra forma de apresentação da Estrutura do Dataframe df
root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Siblings/Spouses Aboard: integer (nullable = true)
 |-- Parents/Children Aboard: integer (nullable = true)
 |-- Fare: double (nullable = true)



In [12]:
# Apresentação das colunas do Dataframe df.

print('Apresentação das colunas do Dataframe df')
df.columns

Apresentação das colunas do Dataframe df


['Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'Siblings/Spouses Aboard',
 'Parents/Children Aboard',
 'Fare']

In [13]:
# Apresentação da quantidade de linhas do Dataframe df.

print('Apresentação da quantidade de linhas do Dataframe df')
df.count()

Apresentação da quantidade de linhas do Dataframe df


887

In [14]:
# Apresentação da Descrição do Dataframe df.

print('Apresentação da Descrição do Dataframe df')
df.describe().show()

Apresentação da Descrição do Dataframe df
+-------+-------------------+------------------+--------------------+------+------------------+-----------------------+-----------------------+-----------------+
|summary|           Survived|            Pclass|                Name|   Sex|               Age|Siblings/Spouses Aboard|Parents/Children Aboard|             Fare|
+-------+-------------------+------------------+--------------------+------+------------------+-----------------------+-----------------------+-----------------+
|  count|                887|               887|                 887|   887|               887|                    887|                    887|              887|
|   mean| 0.3855693348365276| 2.305524239007892|                null|  null|29.471443066516347|     0.5253664036076663|     0.3833145434047351|32.30542018038328|
| stddev|0.48700411775101266|0.8366620036697728|                null|  null|14.121908405462552|      1.104668553867569|     0.8074659070316833|49.78

In [17]:
# Apresentação da quantidade de sobreviventes do Dataframe df.

print('Apresentação da quantidade de sobreviventes do Dataframe df')
print('0 - Não Sobreviveu')
print('1 - Sobreviveu')

df.groupBy('Survived').count().orderBy('Survived').show()

Apresentação da quantidade de sobreviventes do Dataframe df
0 - Não Sobreviveu
1 - Sobreviveu
+--------+-----+
|Survived|count|
+--------+-----+
|       0|  545|
|       1|  342|
+--------+-----+



In [18]:
# Criação da variável indexer, convertendo a coluna 'Sex' de String para Int, através da instância StringIndexer.

print('Criação da variável indexer, convertendo a coluna Sex de String para Int, através da instância StringIndexer')
indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex')

Criação da variável indexer, convertendo a coluna Sex de String para Int, através da instância StringIndexer


In [19]:
# Criação de um novo Dataframe output_fixed com a inclusão da coluna SexIndex.

print('Criação de um novo Dataframe output_fixed com a inclusão da coluna SexIndex')
output_fixed = indexer.fit(df).transform(df)

Criação de um novo Dataframe output_fixed com a inclusão da coluna SexIndex


In [20]:
# Apresentação da estrutura do Dataframe output_fixed.

print('Apresentação da estrutura do Dataframe output_fixed')
output_fixed.printSchema()

Apresentação da estrutura do Dataframe output_fixed
root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Siblings/Spouses Aboard: integer (nullable = true)
 |-- Parents/Children Aboard: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- SexIndex: double (nullable = false)



In [21]:
# Apresentação das 20 primeiras linhas do Dataframe output_fixed.

print('Apresentação das 20 primeiras linhas do Dataframe output_fixed')
output_fixed.show()

Apresentação das 20 primeiras linhas do Dataframe output_fixed
+--------+------+--------------------+------+----+-----------------------+-----------------------+-------+--------+
|Survived|Pclass|                Name|   Sex| Age|Siblings/Spouses Aboard|Parents/Children Aboard|   Fare|SexIndex|
+--------+------+--------------------+------+----+-----------------------+-----------------------+-------+--------+
|       0|     3|Mr. Owen Harris B...|  male|22.0|                      1|                      0|   7.25|     0.0|
|       1|     1|Mrs. John Bradley...|female|38.0|                      1|                      0|71.2833|     1.0|
|       1|     3|Miss. Laina Heikk...|female|26.0|                      0|                      0|  7.925|     1.0|
|       1|     1|Mrs. Jacques Heat...|female|35.0|                      1|                      0|   53.1|     1.0|
|       0|     3|Mr. William Henry...|  male|35.0|                      0|                      0|   8.05|     0.0|
|       0

In [22]:
# Criação da variável myAssembler, criando a coluna features de vetores, dos dados relevantes para a análise da previção
# de sobrevientes do Titanic , através da instância VectorAssembler.

print('Criação da variável myAssembler, Criação da Coluna features - coluna de vetores')
myAssembler = VectorAssembler(inputCols=['Pclass',
 'Age',
 'Siblings/Spouses Aboard',
 'Parents/Children Aboard',
 'Fare',
 'SexIndex'],outputCol='features')

Criação da variável myAssembler, Criação da Coluna features - coluna de vetores


In [23]:
# Criação do Dataframe output com os dados do Datafrma output_fixed e a inclusão da coluna features.

print('Criação do Dataframe output')
output = myAssembler.transform(output_fixed)


Criação do Dataframe output


In [24]:
# Apresentação das 20 primeiras linhas do Dataframe output.

print('Apresentação das 20 primeiras linhas do Dataframe output')
output.show()

Apresentação das 20 primeiras linhas do Dataframe output
+--------+------+--------------------+------+----+-----------------------+-----------------------+-------+--------+--------------------+
|Survived|Pclass|                Name|   Sex| Age|Siblings/Spouses Aboard|Parents/Children Aboard|   Fare|SexIndex|            features|
+--------+------+--------------------+------+----+-----------------------+-----------------------+-------+--------+--------------------+
|       0|     3|Mr. Owen Harris B...|  male|22.0|                      1|                      0|   7.25|     0.0|[3.0,22.0,1.0,0.0...|
|       1|     1|Mrs. John Bradley...|female|38.0|                      1|                      0|71.2833|     1.0|[1.0,38.0,1.0,0.0...|
|       1|     3|Miss. Laina Heikk...|female|26.0|                      0|                      0|  7.925|     1.0|[3.0,26.0,0.0,0.0...|
|       1|     1|Mrs. Jacques Heat...|female|35.0|                      1|                      0|   53.1|     1.0|[1.0,3

In [25]:
# Criação do Dataframe df_final com a seleção das colunas que servirão de Label e Features.
# Label = Survided e Features = features.

print('Criação do Dataframe df_final com a seleção das colunas que servirão de Label e Features')
df_final = output.select('features', 'Survived')

Criação do Dataframe df_final com a seleção das colunas que servirão de Label e Features


In [26]:
# Apresentação da estrutura do Dataframe df_final.

print('Apresentação da estrutura do Dataframe df_final')
df_final.printSchema()

Apresentação da estrutura do Dataframe df_final
root
 |-- features: vector (nullable = true)
 |-- Survived: integer (nullable = true)



In [27]:
# Apresentação das 20 primeiras linhas do Dataframe df_final.

print('Apresentação das 20 primeiras linhas do Dataframe df_final')

df_final.show()

Apresentação das 20 primeiras linhas do Dataframe df_final
+--------------------+--------+
|            features|Survived|
+--------------------+--------+
|[3.0,22.0,1.0,0.0...|       0|
|[1.0,38.0,1.0,0.0...|       1|
|[3.0,26.0,0.0,0.0...|       1|
|[1.0,35.0,1.0,0.0...|       1|
|[3.0,35.0,0.0,0.0...|       0|
|[3.0,27.0,0.0,0.0...|       0|
|[1.0,54.0,0.0,0.0...|       0|
|[3.0,2.0,3.0,1.0,...|       0|
|[3.0,27.0,0.0,2.0...|       1|
|[2.0,14.0,1.0,0.0...|       1|
|[3.0,4.0,1.0,1.0,...|       1|
|[1.0,58.0,0.0,0.0...|       1|
|[3.0,20.0,0.0,0.0...|       0|
|[3.0,39.0,1.0,5.0...|       0|
|[3.0,14.0,0.0,0.0...|       0|
|[2.0,55.0,0.0,0.0...|       1|
|[3.0,2.0,4.0,1.0,...|       0|
|[2.0,23.0,0.0,0.0...|       1|
|[3.0,31.0,1.0,0.0...|       0|
|[3.0,22.0,0.0,0.0...|       1|
+--------------------+--------+
only showing top 20 rows



In [28]:
# Criação dos Dataframes para Treino e Testes.
# Treino com 70% do Dataframe df_final.
# Teste com 30% do Dataframe df_final.

print('Criação dos Dataframes para Treino com 70% do Dataframe df_final e Testes com 30% do Dataframe df_final')
treino_data, teste_data = df_final.randomSplit([0.7, 0.3])

Criação dos Dataframes para Treino com 70% do Dataframe df_final e Testes com 30% do Dataframe df_final


In [29]:
# Criação das Variáveis dtc (Árvore de Decisão) e rfc (Floresta Randômica).

print('Criação das Variáveis dtc (Árvore de Decisão) e rfc (Floresta Randômica)')
dtc = DecisionTreeClassifier(labelCol='Survived', featuresCol='features')
rfc = RandomForestClassifier(labelCol='Survived', featuresCol='features')

Criação das Variáveis dtc (Árvore de Decisão) e rfc (Floresta Randômica)


In [30]:
# Criação das Variáveis dtc_modelo (Árvore de Decisão) e rfc_modelo (Floresta Randômica), para realização do treino 
# dos algoritmos.

print('Criação das Variáveis dtc_modelo (Árvore de Decisão) e rfc_modelo (Floresta Randômica)')
dtc_modelo = dtc.fit(treino_data)
rfc_modelo = rfc.fit(treino_data)

Criação das Variáveis dtc_modelo (Árvore de Decisão) e rfc_modelo (Floresta Randômica)


In [31]:
# Criação das variaveis dtc_preds (Árvore de Decisão) e rfc_preds (Floresta Randômica) para predição dos sobreviventes do Titanic.

print('Criação das variaveis dtc_preds (Árvore de Decisão) e rfc_preds (Floresta Randômica) para predição')
dtc_preds = dtc_modelo.transform(teste_data)
rfc_preds = rfc_modelo.transform(teste_data)

Criação das variaveis dtc_preds (Árvore de Decisão) e rfc_preds (Floresta Randômica) para predição


In [32]:
# Apresentação do Resultado da predição da variável dtc_preds(Árvore de Decisão).

print('Apresentação do Resultado da predição da variável dtc_preds(Árvore de Decisão)')
dtc_preds.show()

Apresentação do Resultado da predição da variável dtc_preds(Árvore de Decisão)
+--------------------+--------+-------------+--------------------+----------+
|            features|Survived|rawPrediction|         probability|prediction|
+--------------------+--------+-------------+--------------------+----------+
|(6,[0,1],[1.0,30.0])|       0|   [7.0,13.0]|         [0.35,0.65]|       1.0|
|(6,[0,1],[1.0,38.0])|       0|  [37.0,13.0]|         [0.74,0.26]|       0.0|
|(6,[0,1],[2.0,19.0])|       0| [264.0,38.0]|[0.87417218543046...|       0.0|
|(6,[0,1],[2.0,37.0])|       0| [264.0,38.0]|[0.87417218543046...|       0.0|
|[1.0,11.0,1.0,2.0...|       1|    [7.0,2.0]|[0.77777777777777...|       0.0|
|[1.0,16.0,0.0,0.0...|       1|  [6.0,106.0]|[0.05357142857142...|       1.0|
|[1.0,17.0,0.0,2.0...|       1|    [7.0,2.0]|[0.77777777777777...|       0.0|
|[1.0,17.0,1.0,0.0...|       1|  [6.0,106.0]|[0.05357142857142...|       1.0|
|[1.0,17.0,1.0,0.0...|       1|  [6.0,106.0]|[0.05357142857142.

In [33]:
# Apresentação do Resultado da predição da variável rfc_preds(Floresta Randômica).

print('Apresentação do Resultado da predição da variável rfc_preds(Floresta Randômica)')
rfc_preds.show()

Apresentação do Resultado da predição da variável rfc_preds(Floresta Randômica)
+--------------------+--------+--------------------+--------------------+----------+
|            features|Survived|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+----------+
|(6,[0,1],[1.0,30.0])|       0|[16.9495895495539...|[0.84747947747769...|       0.0|
|(6,[0,1],[1.0,38.0])|       0|[16.9292491593505...|[0.84646245796752...|       0.0|
|(6,[0,1],[2.0,19.0])|       0|[17.3812965820289...|[0.86906482910144...|       0.0|
|(6,[0,1],[2.0,37.0])|       0|[18.1068547576464...|[0.90534273788232...|       0.0|
|[1.0,11.0,1.0,2.0...|       1|[8.59476234042148...|[0.42973811702107...|       1.0|
|[1.0,16.0,0.0,0.0...|       1|[1.38520470898854...|[0.06926023544942...|       1.0|
|[1.0,17.0,0.0,2.0...|       1|[13.2379475023633...|[0.66189737511816...|       0.0|
|[1.0,17.0,1.0,0.0...|       1|[0.84165925685052...|[0.04208296284252.

In [34]:
# Criação da variável my_binary_eval para levantamento do percentual de acertos do algoritmo.

print('Criação da variável my_binary_eval para levantamento do percentual de acertos do algoritmo')
my_binary_eval = BinaryClassificationEvaluator(labelCol='Survived')

Criação da variável my_binary_eval para levantamento do percentual de acertos do algoritmo


In [35]:
# Apresentação da Avaliação do Resultado do Algoritmo Machine Learning Arvore de Decisão - Decision Tree

print('Imprimindo a Avaliação do Resultado do Algoritmo Machine Learning Arvore de Decisão - Decision Tree')
print(my_binary_eval.evaluate(dtc_preds))

Imprimindo a Avaliação do Resultado do Algoritmo Machine Learning Arvore de Decisão - Decision Tree
0.7975460122699385


In [36]:
# Apresentação da Avaliação do Resultado do Algoritmo Machine Learning Floresta Randômica - Random Forrest

print('Imprimindo a Avaliação do Resultado do Algoritmo Machine Learning Floresta Randômica - Random Forrest')
print(my_binary_eval.evaluate(rfc_preds))

Imprimindo a Avaliação do Resultado do Algoritmo Machine Learning Floresta Randômica - Random Forrest
0.9068860730462703


In [37]:
#Aumentando o número de arvores de 20 para 300.

rfc = RandomForestClassifier(numTrees=300, labelCol='Survived', featuresCol='features')
rfc_modelo = rfc.fit(treino_data)
rfc_preds = rfc_modelo.transform(teste_data)
print('Imprimindo a Avaliação do Resultado do Algoritmo Machine Learning Floresta Randômica - 300 árvores')
print(my_binary_eval.evaluate(dtc_preds))

Imprimindo a Avaliação do Resultado do Algoritmo Machine Learning Floresta Randômica - 300 árvores
0.7975460122699385
